<a href="https://colab.research.google.com/github/elfliska/databaze/blob/main/JupyterNotebook/SQL/Join_joinTriTabulek_zadani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propojování tabulek
Často chceme v dotazu SELECT zobrazit zároveň data z více tabulek.

K propojení tabulek se používá klíčové slovo JOIN, které vytvoří ze dvou či více tabulek jednu velkou (se SLOUPEČKY ze všech vstupních tabulek)

Je několik typů JOINů (podle toho, které ŘÁDKY chceme ve výsledku), my se však budeme zabívat pouze čtyřmi.

A to:
* INNER JOIN
* LEFT JOIN
* RIGHT JOIN
* FULL OUTER JOIN

Než si vysvětlíme jak JOINy fungují musíme si vytvořit tabulky, na kterých si to předvedeme.



In [1]:
! pip install mysql.connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 90.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp312-cp312-linux_x86_64.whl size=247950 sha256=e407cf75abf04c4b1546f8702a2c3185f083972369b66bc58246c164d599ea4f
  Stored in directory: /root/.cache/pip/wheels/03/17/fa/d7604c72dd3dd6d3eb3d249abf36cc532c9a9b4354b8f1bc4f
Successfully built mysql.connector


In [2]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "student11"
    ,password = "spsnet"
    ,database = "vyuka11"
)

mycursor = mydb.cursor()

try:
    mycursor.execute("""DROP TABLE Tym;""")
    mydb.commit()
except mysql.connector.Error:
    print("Tabulka Tym neexistuje.")

try:
    mycursor.execute("""DROP TABLE Hrac;""")
    mydb.commit()
except mysql.connector.Error:
    print("Tabulka Hrac neexistuje.")

try:
    mycursor.execute("""DROP TABLE Pozice;""")
    mydb.commit()
except mysql.connector.Error:
    print("Tabulka Pozice neexistuje.")

# Vytvoříme si potřebné tabulky
mycursor.execute("""CREATE TABLE Tym
(
	id int AUTO_INCREMENT PRIMARY KEY,
	nazev char(20) NOT NULL,
	mesto char(20) NOT NULL,
	zustatek int DEFAULT 0,
	CONSTRAINT UNIKATNI_tym UNIQUE (nazev, mesto) /* UNIQUE na nazev a mesto zaroveň */
);""")

mydb.commit()

mycursor.execute("""CREATE TABLE Pozice(
	id_poz int PRIMARY KEY AUTO_INCREMENT,
	pojmenovani text NOT NULL
);""")

mydb.commit()

mycursor.execute("""CREATE TABLE Hrac(
	id int AUTO_INCREMENT PRIMARY KEY,
	jmeno text NOT NULL,
	prijmeni text NOT NULL,
	datum_narozeni DATE NOT NULL,
	tym_id int NOT NULL,
	pozice_id int NOT NULL

    /* zde by se hodil ještě FOREIGN KEY, abychom měli jistotu,
    že hráče zařazujeme do existujícího týmu */
);""")

mydb.commit()

# A zapíšeme do nich
mycursor.execute("""INSERT INTO Tym (nazev, mesto, zustatek) VALUES
	("HC Rytíři", "Kladno", 1000),
	("HC Slavie", "Praha", 5000),
	("HC Sparta", "Praha", 6000),
	("HC Sparta", "Brno", 5600);""")

mydb.commit()

mycursor.execute("""INSERT INTO Pozice(pojmenovani) VALUES
	("Utocnik"),
	("Obrana"),
	("Brankar"),
	("Zaloha");""")

mydb.commit()

mycursor.execute("""INSERT INTO Hrac (jmeno, prijmeni, datum_narozeni, tym_id, pozice_id) VALUES
	("Adam", "Prvni", '1996-06-02', 2, 2),
	("Pavel", "Druhy", '2000-09-09', 1, 3),
	("Kuba", "Treti", '1998-01-05', 3, 1),
	("Josef", "Čtvrtý", '1997-01-06', 6, 2),
	("Jirka", "Pátý", '1999-03-08', 2, 6);""")

mydb.commit()

mycursor.close()
mydb.close()

## INNER JOIN
INNER JOIN je v MySQL databázích nastaven jako defaultní JOIN.

Ve výsledku tohoto spojení budou pouze řádky, které mají porovnávanou hodnotu vyplněnou v obou tabulkách.

"Porovnávané hodnoty" se získávají ze dvou sloupečků (jeden z každé tabulky) a které sloupečky to budou, určujeme my v našem dotazu. Pokud hodnoty splňují zadanou podmínku, SŘBD spojí tyto dva řádky (z jedné a druhé tabulky) do jednoho dlouhého řádku

Kód pro použití INNER JOINu může vypadat takto:

In [3]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "student11"
    ,password = "spsnet"
    ,database = "vyuka11"
)
mycursor = mydb.cursor()

# bez zadané podmínky a sloupečků s hodnotami k porovnání
mycursor.execute("""SELECT prijmeni, datum_narozeni, nazev, mesto FROM Hrac INNER JOIN Tym;
""") # Tento příkaz vypíše kombinace každého hráče s každým týmem.

myresult = mycursor.fetchall()

for prijmeni, datum_narozeni, nazev, mesto in myresult:
    print(f"Hráč {prijmeni}, s datem narození {datum_narozeni} z týmu {nazev} v městě {mesto}")

print("=========")
# Tento JOIN je nám však k ničemu, protože neposkytuje žádné informace
# Takže k příkazu tedy přidáme podmínku,
# tak aby spolili pouze řádky, kde hrac.tym_id =  tym.id
mycursor.execute("""SELECT prijmeni, datum_narozeni, nazev, mesto FROM Hrac JOIN Tym
    ON Hrac.tym_id = Tym.id;
""")

myresult = mycursor.fetchall()

for prijmeni, datum_narozeni, nazev, mesto in myresult:
    print(f"Hráč {prijmeni}, s datem narození {datum_narozeni} z týmu {nazev} v městě {mesto}")
# Jak můžete vidět, tak hráč s příjmením Čtvrtý se nevypsal,
# protože k jeho tym_id není odpovídající id v tabulce Tym

mycursor.close()
mydb.close()

Hráč Prvni, s datem narození 1996-06-02 z týmu HC Rytíři v městě Kladno
Hráč Prvni, s datem narození 1996-06-02 z týmu HC Slavie v městě Praha
Hráč Prvni, s datem narození 1996-06-02 z týmu HC Sparta v městě Brno
Hráč Prvni, s datem narození 1996-06-02 z týmu HC Sparta v městě Praha
Hráč Druhy, s datem narození 2000-09-09 z týmu HC Rytíři v městě Kladno
Hráč Druhy, s datem narození 2000-09-09 z týmu HC Slavie v městě Praha
Hráč Druhy, s datem narození 2000-09-09 z týmu HC Sparta v městě Brno
Hráč Druhy, s datem narození 2000-09-09 z týmu HC Sparta v městě Praha
Hráč Treti, s datem narození 1998-01-05 z týmu HC Rytíři v městě Kladno
Hráč Treti, s datem narození 1998-01-05 z týmu HC Slavie v městě Praha
Hráč Treti, s datem narození 1998-01-05 z týmu HC Sparta v městě Brno
Hráč Treti, s datem narození 1998-01-05 z týmu HC Sparta v městě Praha
Hráč Čtvrtý, s datem narození 1997-01-06 z týmu HC Rytíři v městě Kladno
Hráč Čtvrtý, s datem narození 1997-01-06 z týmu HC Slavie v městě Praha
Hrá

## LEFT JOIN
Jediný rozdíl od INNER JOINu je v tom jak SŘBD pracuje s chybějícími položkami.
LEFT JOIN vezme všechny záznamy z tabulky nalevo a pokud k nim chybí zápis napravo, tak doplní NULL.

Kód pro použití LEFT JOINu může vypadat takto:

In [4]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "student11"
    ,password = "spsnet"
    ,database = "vyuka11"
)
mycursor = mydb.cursor()

# LEFT JOIN výsledek uzná, i pokud existuje pouze levá část vazby (zde Tym)
# a pravá (ta připojovaná, zde Hrac) neexistuje. Do hodnot sloupců z připojované části se vloží NULL.
# Levá a pravá část vazby se rozlišuje podle toho kde se název tabulky nachází vzhledem k JOINu.
mycursor.execute("""SELECT prijmeni, datum_narozeni, nazev, mesto FROM
    Tym LEFT JOIN Hrac ON Tym.id=Hrac.tym_id;
""")

myresult = mycursor.fetchall()

for prijmeni, datum_narozeni, nazev, mesto in myresult:
    print(f"Hráč {prijmeni}, s datem narození {datum_narozeni} z týmu {nazev} v městě {mesto}")

mycursor.close()
mydb.close()

Hráč Prvni, s datem narození 1996-06-02 z týmu HC Slavie v městě Praha
Hráč Druhy, s datem narození 2000-09-09 z týmu HC Rytíři v městě Kladno
Hráč Treti, s datem narození 1998-01-05 z týmu HC Sparta v městě Praha
Hráč Pátý, s datem narození 1999-03-08 z týmu HC Slavie v městě Praha
Hráč None, s datem narození None z týmu HC Sparta v městě Brno


## RIGHT JOIN
Jediný rozdíl od INNER JOINu je v tom jak SŘBD pracuje s chybějícími položkami.
RIGHT JOIN vezme všechny záznamy z tabulky napravo a pokud k nim chybí zápis nalevo, tak doplní NULL.

Kód pro použití LEFT JOINu může vypadat takto:

In [5]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "student11"
    ,password = "spsnet"
    ,database = "vyuka11"
)
mycursor = mydb.cursor()

# RIGHT JOIN výsledek uzná, i pokud existuje pouze pravá část vazby (zde Hrac)
# a levá (ta připojovaná, zde Tym) neexistuje. Do hodnot sloupců z připojované části se vloží NULL.
# Levá a pravá část vazby se rozlišuje podle toho kde se název tabulky nachází vzheledem k JOINu.
mycursor.execute("""SELECT prijmeni, datum_narozeni, nazev, mesto FROM
    Tym RIGHT JOIN Hrac ON Tym.id=Hrac.tym_id;
""")

myresult = mycursor.fetchall()

for prijmeni, datum_narozeni, nazev, mesto in myresult:
    print(f"Hráč {prijmeni}, s datem narození {datum_narozeni} z týmu {nazev} v městě {mesto}")

mycursor.close()
mydb.close()

Hráč Prvni, s datem narození 1996-06-02 z týmu HC Slavie v městě Praha
Hráč Druhy, s datem narození 2000-09-09 z týmu HC Rytíři v městě Kladno
Hráč Treti, s datem narození 1998-01-05 z týmu HC Sparta v městě Praha
Hráč Čtvrtý, s datem narození 1997-01-06 z týmu None v městě None
Hráč Pátý, s datem narození 1999-03-08 z týmu HC Slavie v městě Praha


## FULL OUTER JOIN
FULL OUTER JOIN vypíše všechny záznamy z levé tabulky i všechny záznamy z pravé tabulky - ty které k sobě pasují samozřejmě spojí.
V podstatě se jedná o kombinaci LEFT JOINu a RIGHT JOINu.
Tento JOIN obvykle vrací velké množství záznamů, které obsahují všechny hodnoty z obou tabulek.

__pozn.:__ MySQL nepodporuje syntaxy pro FULL OUTER JOIN, takže ho musíme trochu obejít.<br>
To se udělá tak, že vytvoříme LEFT JOIN a RIGHT JOIN a použijeme příkaz UNION.<br>
Příkaz UNION "slepí" tyto 2 výsledky (z obou JOINů dohromady (pod sebe)). Tím se tedy vytvoří logika FULL OUTER JOINu.

Kód pro použití FULL OUTER JOINu může vypadat takto:

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

mycursor.execute("""SELECT prijmeni, datum_narozeni, nazev, mesto
    FROM Hrac LEFT JOIN Tym ON Hrac.tym_id = Tym.id
    UNION
    SELECT prijmeni, datum_narozeni, nazev, mesto
    FROM Hrac RIGHT JOIN Tym ON Hrac.tym_id = Tym.id;
""")

myresult = mycursor.fetchall()

for prijmeni, datum_narozeni, nazev, mesto in myresult:
    print(f"Hráč {prijmeni}, s datem narození {datum_narozeni} z týmu {nazev} v městě {mesto}")

mycursor.close()
mydb.close()

## JOIN tří tabulek
JOIN tří tabulek je v podstatě stejný jako JOIN dvou tabulek, jenom se do příkazu přidá další JOIN.
Spojování více tabulek se hodí pokud máme několik tabulek, které na sebe odkazují pouze přes např. id a chceme informace z těchto tabulek spojit.

Kód pro spojení tří tabulek může vypadat takto:

In [8]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "student11"
    ,password = "spsnet"
    ,database = "vyuka11"
)
mycursor = mydb.cursor()

mycursor.execute("""SELECT nazev, mesto, prijmeni, pojmenovani
    FROM Tym JOIN Hrac ON Tym.id = Hrac.tym_id JOIN Pozice ON Hrac.pozice_id = Pozice.id_poz;
""")

myresult = mycursor.fetchall()

for nazev, mesto, prijmeni, pojmenovani in myresult:
    print(f"Hráč {prijmeni} z týmu {nazev} v městě {mesto} má pozici {pojmenovani}")

mycursor.close()
mydb.close()

Hráč Treti z týmu HC Sparta v městě Praha má pozici Utocnik
Hráč Prvni z týmu HC Slavie v městě Praha má pozici Obrana
Hráč Druhy z týmu HC Rytíři v městě Kladno má pozici Brankar


# Cvičení
Bude následovat několik úkolů na procvičení

## Cvičení 1:
Přihlašte se do školní databáze a vypište záznamy kde Hrac.pozice_id=Pozice.id_poz.
Vypište pouze záznamy, které nebudou obsahovat NULL.

Vypište atributy jmeno, prijmeni, pojmenovani.

In [11]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "student11"
    ,password = "spsnet"
    ,database = "vyuka11"
)
mycursor = mydb.cursor()

# TODO: Zde přijde váš kód ->
mycursor.execute("""SELECT jmeno, prijmeni, pojmenovani FROM
    Hrac JOIN Pozice ON Hrac.pozice_id=Pozice.id_poz;
""")

myresult = mycursor.fetchall()

for jmeno, prijmeni, pojmenovani in myresult:
    print(f"Hráč {jmeno} {prijmeni}, z pozicí {pojmenovani}")


mycursor.close()
mydb.close()

Hráč Adam Prvni, z pozicí Obrana
Hráč Pavel Druhy, z pozicí Brankar
Hráč Kuba Treti, z pozicí Utocnik
Hráč Josef Čtvrtý, z pozicí Obrana


### Referenční řešení pro kontrolu:
<pre>
Hráč Kuba Treti má pozici Utocnik
Hráč Adam Prvni má pozici Obrana
Hráč Josef Čtvrtý má pozici Obrana
Hráč Pavel Druhy má pozici Brankar
</pre>

## Cvičení 2:
Přihlašte se do školní databáze a vypište všechny hráče. U těch, kteří mají přidělenou pozici vypište i tuto pozici.

Vypište atributy jmeno, prijmeni, pozice.

In [12]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "student11"
    ,password = "spsnet"
    ,database = "vyuka11"
)
mycursor = mydb.cursor()

# TODO: Zde přijde váš kód ->
mycursor.execute("""SELECT jmeno, prijmeni, pojmenovani FROM
    Hrac LEFT JOIN Pozice ON Hrac.pozice_id=Pozice.id_poz;
""")

myresult = mycursor.fetchall()

for jmeno, prijmeni, pojmenovani in myresult:
    print(f"Hráč {jmeno} {prijmeni}, z pozicí {pojmenovani}")


mycursor.close()
mydb.close()

Hráč Adam Prvni, z pozicí Obrana
Hráč Pavel Druhy, z pozicí Brankar
Hráč Kuba Treti, z pozicí Utocnik
Hráč Josef Čtvrtý, z pozicí Obrana
Hráč Jirka Pátý, z pozicí None


### Referenční řešení pro kontrolu:
<pre>
Hráč Adam Prvni má pozici Obrana
Hráč Pavel Druhy má pozici Brankar
Hráč Kuba Treti má pozici Utocnik
Hráč Josef Čtvrtý má pozici Obrana
Hráč Jirka Pátý má pozici None
</pre>

## Cvičení 3:
Přihlašte se do školní databáze a vypište všechny Pozice. Poizce které jsou obsazené hráčem u sebe budou mít napsaneho hráče, který má danou poizici.

Vypište atributy jmeno, prijmeni, pozice.

In [13]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "student11"
    ,password = "spsnet"
    ,database = "vyuka11"
)
mycursor = mydb.cursor()

# TODO: Zde přijde váš kód ->
mycursor.execute("""SELECT jmeno, prijmeni, pojmenovani FROM
    Hrac RIGHT JOIN Pozice ON Hrac.pozice_id=Pozice.id_poz;
""")

myresult = mycursor.fetchall()

for jmeno, prijmeni, pojmenovani in myresult:
    print(f"Hráč {jmeno} {prijmeni}, z pozicí {pojmenovani}")

mycursor.close()
mydb.close()

Hráč Adam Prvni, z pozicí Obrana
Hráč Pavel Druhy, z pozicí Brankar
Hráč Kuba Treti, z pozicí Utocnik
Hráč Josef Čtvrtý, z pozicí Obrana
Hráč None None, z pozicí Zaloha


### Referenční řešení pro kontrolu:
<pre>
Hráč Adam Prvni má pozici Obrana
Hráč Pavel Druhy má pozici Brankar
Hráč Kuba Treti má pozici Utocnik
Hráč Josef Čtvrtý má pozici Obrana
Hráč None None má pozici Zaloha
</pre>

## Cvičení 4:
Přihlašte se do školní databáze a vypište všechny hráče a všechny pozice. Pokud má hráč přiřazenou pozici (nebo pozice hráče) bude ve výsledku tento vztah poznat.

Vypište všechny atributy.

In [19]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "student11"
    ,password = "spsnet"
    ,database = "vyuka11"
)
mycursor = mydb.cursor()

# TODO: Zde přijde váš kód ->
mycursor.execute("""SELECT *
      FROM Hrac LEFT JOIN Pozice ON Hrac.pozice_id=Pozice.id_poz
      UNION
      SELECT *
      FROM Hrac RIGHT JOIN Pozice ON Hrac.pozice_id=Pozice.id_poz;
""")

myresult = mycursor.fetchall()

for id, jmeno, prijmeni, datum_narozeni, tym_id, pozice_id, id_poz, pojmenovani in myresult:
    print(f"Hráč s id {id} {jmeno} {prijmeni}, datum narození {datum_narozeni}, hraje v týmu s id {tym_id} na pozici {pozice_id} s pozicí {id_poz} {pojmenovani}")


mycursor.close()
mydb.close()

Hráč s id 1 Adam Prvni, datum narození 1996-06-02, hraje v týmu s id 2 na pozici 2 s pozicí 2 Obrana
Hráč s id 2 Pavel Druhy, datum narození 2000-09-09, hraje v týmu s id 1 na pozici 3 s pozicí 3 Brankar
Hráč s id 3 Kuba Treti, datum narození 1998-01-05, hraje v týmu s id 3 na pozici 1 s pozicí 1 Utocnik
Hráč s id 4 Josef Čtvrtý, datum narození 1997-01-06, hraje v týmu s id 6 na pozici 2 s pozicí 2 Obrana
Hráč s id 5 Jirka Pátý, datum narození 1999-03-08, hraje v týmu s id 2 na pozici 6 s pozicí None None
Hráč s id None None None, datum narození None, hraje v týmu s id None na pozici None s pozicí 4 Zaloha


### Referenční řešení pro kontrolu:
<pre>
Hráč s id 1 Adam Prvni nar. 1996-06-02 hraje v týmu s id 2 na poici s id 2 - 2 Obrana
Hráč s id 2 Pavel Druhy nar. 2000-09-09 hraje v týmu s id 1 na poici s id 3 - 3 Brankar
Hráč s id 3 Kuba Treti nar. 1998-01-05 hraje v týmu s id 3 na poici s id 1 - 1 Utocnik
Hráč s id 4 Josef Čtvrtý nar. 1997-01-06 hraje v týmu s id 6 na poici s id 2 - 2 Obrana
Hráč s id 5 Jirka Pátý nar. 1999-03-08 hraje v týmu s id 2 na poici s id 6 - None None
Hráč s id None None None nar. None hraje v týmu s id None na poici s id None - 4 Zaloha
</pre>
